In [7]:
import os
import numpy as np
import pandas as pd
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from torchvision import transforms, models
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
import torch.nn as nn
import optuna
from sklearn.model_selection import cross_val_score

In [2]:

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

csv_file = "train.csv"
img_dir = "train/train"
df = pd.read_csv(csv_file)

# Calculate class weights for 'jenis' category
class_counts = df['jenis'].value_counts().sort_index().values
class_weights = 1. / torch.tensor(class_counts, dtype=torch.float)

# Create a sampler with the weights
sample_weights = [class_weights[label] for label in df['jenis'].values]
sampler = WeightedRandomSampler(weights=sample_weights, num_samples=len(sample_weights), replacement=True)

# Custom Dataset Class
class CustomDataset(Dataset):
    def __init__(self, csv_file, img_dir, transform=None):
        self.img_labels = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_id = self.img_labels.iloc[idx, 0]  # Image ID
        jenis_label = self.img_labels.iloc[idx, 1]  # Type (0: T-shirt, 1: Hoodie)
        warna_label = self.img_labels.iloc[idx, 2]  # Color

        # Check for image format
        img_path_jpg = os.path.join(self.img_dir, f"{img_id}.jpg")
        img_path_png = os.path.join(self.img_dir, f"{img_id}.png")
        
        if os.path.exists(img_path_jpg):
            img_path = img_path_jpg
        elif os.path.exists(img_path_png):
            img_path = img_path_png
        else:
            raise FileNotFoundError(f"Image {img_id} not found in JPG or PNG format.")
        
        image = Image.open(img_path)
        if self.transform:
            image = self.transform(image)

        return image, jenis_label, warna_label

# Transform
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Dataset and DataLoader
dataset = CustomDataset(csv_file=csv_file, img_dir=img_dir, transform=transform)
dataloader = DataLoader(dataset, batch_size=32, sampler=sampler)

# Custom ResNet Model with Dropout
class ResNetWithDropout(nn.Module):
    def __init__(self, base_model, dropout_rate=0.5):
        super(ResNetWithDropout, self).__init__()
        self.base_model = base_model
        self.dropout = nn.Dropout(p=dropout_rate)

    def forward(self, x):
        x = self.base_model(x)
        x = self.dropout(x)  # Apply dropout after feature extraction
        return x

# Load ResNet18 and wrap it with the custom model
resnet18 = models.resnet18(pretrained=True)
resnet_model = ResNetWithDropout(resnet18)  # Wrap ResNet with dropout
resnet_model.fc = nn.Identity()  # Remove the final classification layer
resnet_model.to(device)

# Unfreeze the last few layers of ResNet18
for param in resnet_model.base_model.parameters():  # Access base model's parameters
    param.requires_grad = False

for param in resnet_model.base_model.layer4.parameters():  # Fine-tuning last layer
    param.requires_grad = True

# Set up optimizer for ResNet18
optimizer = torch.optim.Adam(resnet_model.parameters(), lr=1e-4)  # Adjust learning rate as needed

# Feature extraction function
def extract_features(model, dataloader):
    model.eval()
    features = []
    labels_jenis = []
    labels_warna = []

    with torch.no_grad():
        for images, jenis_labels, warna_labels in dataloader:
            images = images.to(device)
            output = model(images)
            features.extend(output.cpu().numpy())
            labels_jenis.extend(jenis_labels.numpy())
            labels_warna.extend(warna_labels.numpy())
            
    return features, labels_jenis, labels_warna

# Extract features
train_features, train_labels_jenis, train_labels_warna = extract_features(resnet_model, dataloader)

# Encode labels
encoder_jenis = LabelEncoder()
encoder_warna = LabelEncoder()
encoded_labels_jenis = encoder_jenis.fit_transform(train_labels_jenis)
encoded_labels_warna = encoder_warna.fit_transform(train_labels_warna)

# Apply best parameters from Optuna
best_C = 0.006947856070131862
best_kernel = 'linear'

# Update SVM classifiers with the best parameters
svm_classifier_jenis = make_pipeline(StandardScaler(), SVC(C=best_C, kernel=best_kernel))
svm_classifier_warna = make_pipeline(StandardScaler(), SVC(C=best_C, kernel=best_kernel))

# Train SVMs on extracted features with the best parameters
svm_classifier_jenis.fit(train_features, encoded_labels_jenis)
svm_classifier_warna.fit(train_features, encoded_labels_warna)


d:\Hology\.venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\Hology\.venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(C=0.006947856070131862, kernel='linear'))])

In [3]:
# Inference (Example)
def predict(image):
    image = transform(image).unsqueeze(0).to(device)
    with torch.no_grad():
        feature = resnet18(image).cpu().numpy()
    
    jenis_pred = svm_classifier_jenis.predict(feature)
    warna_pred = svm_classifier_warna.predict(feature)
    
    jenis_pred_label = encoder_jenis.inverse_transform(jenis_pred)
    warna_pred_label = encoder_warna.inverse_transform(warna_pred)
    
    return jenis_pred_label[0], warna_pred_label[0]


In [4]:
from sklearn.metrics import accuracy_score

# Predict on training data for both classifiers
pred_labels_jenis = svm_classifier_jenis.predict(train_features)
pred_labels_warna = svm_classifier_warna.predict(train_features)

# Calculate accuracy for jenis
accuracy_jenis = accuracy_score(encoded_labels_jenis, pred_labels_jenis) * 100

# Calculate accuracy for warna
accuracy_warna = accuracy_score(encoded_labels_warna, pred_labels_warna) * 100

# Calculate combined accuracy
combined_correct = (pred_labels_jenis == encoded_labels_jenis) & (pred_labels_warna == encoded_labels_warna)
accuracy_both_correct = accuracy_score(combined_correct, [True] * len(combined_correct)) * 100

# Print the accuracies
print(f"Accuracy (Jenis): {accuracy_jenis:.2f}%")
print(f"Accuracy (Warna): {accuracy_warna:.2f}%")
print(f"Accuracy (Both Correct): {accuracy_both_correct:.2f}%")


Accuracy (Jenis): 99.49%
Accuracy (Warna): 99.74%
Accuracy (Both Correct): 99.23%


Testing

In [9]:
def predict_images(test_folder, device):
    results = []
    
    # Get a sorted list of image files based on numeric part of the file name
    img_files = sorted(
        [img_name for img_name in os.listdir(test_folder) if img_name.endswith(('.jpg', '.png'))],
        key=lambda x: int(os.path.splitext(x)[0])  # Sort by the numeric part of the filename
    )
    
    # Prepare to collect features for the test set
    test_features = []
    img_ids = []

    for img_name in img_files:
        img_id = os.path.splitext(img_name)[0]  # Get the ID from the file name
        img_path = os.path.join(test_folder, img_name)

        # Load and transform the image
        image = Image.open(img_path).convert('RGB')  # Ensure image is in RGB mode
        image = transform(image).unsqueeze(0)  # Add batch dimension
        image = image.to(device)  # Move to the appropriate device

        # Extract features using ResNet18
        with torch.no_grad():
            feature = resnet18(image).cpu().numpy()
            test_features.append(feature)

        img_ids.append(int(img_id))  # Store the ID for results

    # Convert the list of features to a 2D numpy array
    test_features = np.vstack(test_features)

    # Make predictions
    jenis_pred = svm_classifier_jenis.predict(test_features)
    warna_pred = svm_classifier_warna.predict(test_features)

    # Store prediction results
    for img_id, jenis, warna in zip(img_ids, jenis_pred, warna_pred):
        results.append({
            'id': img_id,  # ID for the image
            'jenis': jenis,  # Predicted type (0: T-shirt, 1: Hoodie)
            'warna': warna   # Predicted color (0: red, 1: yellow, 2: blue, 3: black, 4: white)
        })
    
    return results

# Path ke folder gambar test
test_folder = "test/test"  # Ganti dengan path ke folder test Anda

# Melakukan prediksi
predictions = predict_images(test_folder, device)

# Membuat DataFrame dan menyimpan ke CSV
submission_df = pd.DataFrame(predictions)
submission_df.to_csv('submission.csv', index=False)  # Ganti dengan nama file yang diinginkan

print("Submission file 'submission.csv' has been created successfully!")

Submission file 'submission.csv' has been created successfully!


Tuning

In [5]:


# Define the objective function
def objective(trial):
    # Hyperparameters for tuning
    C = trial.suggest_loguniform('C', 1e-3, 1e3)
    kernel = trial.suggest_categorical('kernel', ['linear', 'rbf'])
    
    # Additional parameter if using 'rbf' kernel
    gamma = 'scale' if kernel == 'linear' else trial.suggest_loguniform('gamma', 1e-4, 1e1)
    
    # Create the SVM classifiers with the trial's suggested parameters
    svm_classifier_jenis = make_pipeline(StandardScaler(), SVC(C=C, kernel=kernel, gamma=gamma))
    svm_classifier_warna = make_pipeline(StandardScaler(), SVC(C=C, kernel=kernel, gamma=gamma))
    
    # Perform cross-validation
    jenis_scores = cross_val_score(svm_classifier_jenis, train_features, encoded_labels_jenis, cv=5)
    warna_scores = cross_val_score(svm_classifier_warna, train_features, encoded_labels_warna, cv=5)
    
    # Calculate mean accuracy for each classifier
    mean_accuracy = (jenis_scores.mean() + warna_scores.mean()) / 2
    
    # Return the average accuracy as the objective to maximize
    return mean_accuracy

# Run the optimization
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)

# Get the best parameters
best_params = study.best_params
print("Best parameters found: ", best_params)

# Train with the best parameters
svm_classifier_jenis_best = make_pipeline(StandardScaler(), SVC(**best_params))
svm_classifier_warna_best = make_pipeline(StandardScaler(), SVC(**best_params))

# Fit the classifiers
svm_classifier_jenis_best.fit(train_features, encoded_labels_jenis)
svm_classifier_warna_best.fit(train_features, encoded_labels_warna)

# Evaluate the tuned model on test data or using cross-validation as needed


d:\Hology\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2024-10-26 13:17:49,807] A new study created in memory with name: no-name-e9f28227-6063-43b6-ae93-bd2fed05de8c
C:\Users\lmaos\AppData\Local\Temp\ipykernel_16540\2034694924.py:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform('C', 1e-3, 1e3)
C:\Users\lmaos\AppData\Local\Temp\ipykernel_16540\2034694924.py:11: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  gamma = 'scale' if kernel == 'linear' else trial.sugg

Best parameters found:  {'C': 0.006947856070131862, 'kernel': 'linear'}


Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(C=0.006947856070131862, kernel='linear'))])